In [1]:
!pip install ultralytics
from os import listdir
from os.path import isfile, join

from os import listdir
from PIL import Image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 9.4 MB/s eta 0:00:00


In [2]:
ONLY_REFUGEE_DIR = '/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee'
file_paths = [join(ONLY_REFUGEE_DIR, f) for f in listdir(ONLY_REFUGEE_DIR) if isfile(join(ONLY_REFUGEE_DIR, f))]


def image_valid(file_path):
    try:
        img = Image.open(file_path)  # Open the image file
        img.load()    # Attempt to load the image fully into memory
        img.close()   # Close the image file
        return True
    except (IOError, SyntaxError, OSError, AttributeError) as e:
        print('Problem with file:', file_path, "Error:", e)
        return False

# Assuming file_paths is a list of file paths
invalid_file_paths = [x for x in file_paths if not(image_valid(x))]
valid_file_paths = [x for x in file_paths if image_valid(x)]

print(len(invalid_file_paths) + len(valid_file_paths) == len(file_paths))
print("invalid file example: ", invalid_file_paths[0])
print("num invalid:",len(invalid_file_paths))
print("num valid:", len(valid_file_paths))

Problem with file: /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/11023.jpg Error: image file is truncated (2 bytes not processed)
Problem with file: /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/9281.jpg Error: image file is truncated (290 bytes not processed)
Problem with file: /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/6824.jpg Error: image file is truncated
Problem with file: /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/11023.jpg Error: image file is truncated (2 bytes not processed)
Problem with file: /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/9281.jpg Error: image file is truncated (290 bytes not processed)
Problem with file: /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/6824.jpg Error: image file is truncated
True
invalid file example:  /content/drive/Othercomputers/My MacBook Pro/data/image_201509_onl

In [3]:
from ultralytics import YOLO

model = YOLO('Facial_expression_model.pt')
BATCH_SIZE = 100
results = []
for i in range(0, len(valid_file_paths), BATCH_SIZE):
    results.extend(model(valid_file_paths[i:i+BATCH_SIZE], conf=0.90))



0: 640x640 1 neutral, 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 1 neutral, 5: 640x640 1 neutral, 6: 640x640 1 happy, 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 1 neutral, 14: 640x640 1 neutral, 15: 640x640 (no detections), 16: 640x640 1 neutral, 17: 640x640 (no detections), 18: 640x640 (no detections), 19: 640x640 1 neutral, 20: 640x640 (no detections), 21: 640x640 (no detections), 22: 640x640 (no detections), 23: 640x640 (no detections), 24: 640x640 (no detections), 25: 640x640 (no detections), 26: 640x640 (no detections), 27: 640x640 (no detections), 28: 640x640 1 neutral, 29: 640x640 (no detections), 30: 640x640 (no detections), 31: 640x640 (no detections), 32: 640x640 (no detections), 33: 640x640 (no detections), 34: 640x640 (no detections), 35: 640x640 (no detections), 36: 640x640 (no detec

In [4]:
import numpy as np

data = {}
label_dict = results[0].names
num_keys = len(results[0].names.keys())
labels = [v for k, v in label_dict.items()]

for result in results:
  arr = np.zeros(num_keys)
  for box_label in result.boxes.cls.cpu().numpy().astype(int):
    arr[box_label] += 1
  path = result.path
  data[path] = arr


In [22]:
import pandas as pd
df = pd.DataFrame.from_dict(data, orient='index', columns=labels)
df.to_csv('out.csv')

df[(df["angry"] > 1) | (df["disappointed"] > 1)]

,angry,disappointed,happy,neutral,sad,surprised
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/9916.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/10374.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/10434.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/6589.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/9305.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/8102.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/8755.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/9286.jpg,3.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/9385.jpg,2.0,0.0,0.0,0.0,0.0,0.0
/content/drive/Othercomputers/My MacBook Pro/data/image_201509_onlyrefugee/2091.jpg,2.0,0.0,0.0,0.0,0.0,0.0
